**EDA and Basic Model**

- For EDA I would be using the pandas profiling package (since that does a lot of the EDA for me) and will make changes to individual variable from there. If you have not looked at pandas profiling package then you should because it is a very powerful EDA package
- The next step would be to create a simple GLM model and a RF model
- I will then use the H2O package to run the model
- Compare the results of manual tweaking vs H2O model

In [1]:
import numpy as np
import pandas as pd
import pandas_profiling as pp

#plotly packages
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tools

In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
print('There are ',train.shape[0],'rows and ',train.shape[1],'columns in the train dataset')
print('There are ',test.shape[0],'rows and ',test.shape[1],'columns in the test dataset')

There are  1460 rows and  81 columns in the train dataset
There are  1459 rows and  80 columns in the test dataset


In [4]:
train.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
print(list(train.columns))

['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC'

## Looking at the Target Variable

In [6]:
data = go.Histogram(x=train.SalePrice)
fig = [data]
py.iplot(fig)

Lets try the log of the target varirable

In [7]:
data = go.Histogram(x=np.log(train.SalePrice))
fig = [data]
py.iplot(fig)

In [8]:
#Saving the log_price variables
SalesPrice_log = np.log(train.SalePrice)
SalePrice = train.SalePrice
Id = train.Id

In [9]:
#Removing the ID and SalePrice Variable
train = train.drop(['Id','SalePrice'], 1)
test = test.drop(['Id'], 1)
print(train.shape)
print(test.shape)

(1460, 79)
(1459, 79)


In [10]:
#Combining the train and testing dataset
train_test = pd.concat([train,test], axis=0, sort=False)
train_test.shape

(2919, 79)

# Removing Extreme Values from Train Dataset and Identifying Columns to Remove

## Splitting the data into numerical and categorical

### Numerical data

In [11]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_num = train.select_dtypes(include=numerics)

### Categorical data

In [12]:
columns_to_rem =  train_num.columns.tolist()
train_cat = train[train.columns.difference(columns_to_rem)]

#### Removing outliers from numerical data using z-score and IQR (Interquartile Range) 

In [13]:
def missing_values(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    return missing_value_df

missing_values(train_num)

,column_name,percent_missing
MSSubClass,MSSubClass,0.000000
LotFrontage,LotFrontage,17.739726
LotArea,LotArea,0.000000
OverallQual,OverallQual,0.000000
OverallCond,OverallCond,0.000000
YearBuilt,YearBuilt,0.000000
YearRemodAdd,YearRemodAdd,0.000000
MasVnrArea,MasVnrArea,0.547945
BsmtFinSF1,BsmtFinSF1,0.000000
BsmtFinSF2,BsmtFinSF2,0.000000


In [14]:
#Filling the missing values with mean
train_num = train_num.fillna(train_num.mean())

In [15]:
from scipy.stats import zscore

z = train_num.apply(zscore)
threshold = 3
np.where(z > 10)

(array([   5,   55,   88,  170,  185,  197,  205,  249,  313,  335,  346,
         635,  706,  810,  934, 1009, 1170, 1182, 1230, 1298, 1298, 1298,
        1298, 1386, 1423, 1437]),
 array([30, 30, 14, 14, 14, 32, 30,  2,  2,  2, 33, 14,  2, 32,  1, 14, 32,
        32, 33,  1,  8, 11, 32, 32, 32, 30]))

In [16]:
Q1 = train_num.quantile(0.25)
Q3 = train_num.quantile(0.75)
IQR = Q3 - Q1

In [17]:
print(train_num < (Q1 - 1.5 * IQR)) |(train_num > (Q3 + 1.5 * IQR).any())

      MSSubClass  LotFrontage  LotArea   ...    MiscVal  MoSold  YrSold
0          False        False    False   ...      False   False   False
1          False        False    False   ...      False   False   False
2          False        False    False   ...      False   False   False
3          False        False    False   ...      False   False   False
4          False        False    False   ...      False   False   False
5          False        False    False   ...      False   False   False
6          False        False    False   ...      False   False   False
7          False        False    False   ...      False   False   False
8          False        False    False   ...      False   False   False
9          False        False    False   ...      False   False   False
10         False        False    False   ...      False   False   False
11         False        False    False   ...      False   False   False
12         False        False    False   ...      False   False 

TypeError: Could not compare [None] with block values

# Normalizing values based on Box-Cox Transformation

In [18]:
profile = pp.ProfileReport(train_num)

In [19]:
profile

Number of variables,36
Number of observations,1460
Total Missing (%),0.0%
Total size in memory,410.7 KiB
Average record size in memory,288.1 B
Numeric,36
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


**Variable**: 1stFlrSF  
**Comments**: Looks like the distribution is not normal. I will try to use the log and compare the before and after  
**Findings**: The distribution is a lot more normalized with the log

In [20]:
pd.set_option('mode.chained_assignment', None)
train['1stFlrSF_log'] = np.log(train['1stFlrSF'])

In [21]:
#Function to make a Histogram Plot

In [22]:
trace0 = go.Histogram(
    x=train['1stFlrSF_log'],
    name='1stFlrSF_log',
    nbinsx = 100,
    marker=dict(
        color='grey',
        line = dict(
            color = 'black',
            width = 0.5
          ),
    ),
    opacity = 0.70,
  )
trace1 = go.Histogram(
    x=train['1stFlrSF'],
    name='1stFlrSF',
    nbinsx = 100,
    marker=dict(
        color='blue',
        line = dict(
            color = 'black',
            width = 0.5
          ),
    ),
    opacity = 0.70
  )
  
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('1stFlrSF_log','1stFlrSF'))
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



**Variable**: 2ndFlrSF  
**Comments**: There are a lot of zeros in here. Lets check what those actual zeros mean  
**Findings**: There are a lot of zeros for House Style with 2Story which means that they are not actually zero but null values. Will change them into Nulls

In [23]:
train['2ndFlrSF_new'] = np.where(train['2ndFlrSF']>=0, train['2ndFlrSF'], -1)

**Variable**: 3SsnPorch  
**Comments**: ~98% of the values are zeros. Probably best to remove this feature  
**Findings**: Create a list for columns to remove

In [24]:
col_to_rem = ['3SsnPorch']

**Variable**: BedroomAbvGr  
**Comments**: ~0.4% of the values are zeros. I will keep it as it is

**Variable**: BsmtFinSF1  
**Comments**: ~32.0% are zeros and there are also some extreme values (>2000)  
**Findings**: Looks like there are zeros whererever the BsmtFinType1 and BsmtFinType2 is Unf (Unfinished). These basement sqft is not actually zero. I would have to convert it to -1.
I will also remove rows greater than 2000 (4 rows)

In [25]:
#Lets check how the data looks like when BsmtFinSF1 is zero for columns that contain Bsmt
bsmt_cols = [col for col in train.columns if 'Bsmt' in col]
train_bsmt = train[bsmt_cols][train['BsmtFinSF1']== 0]
train_bsmt.head(10)


,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath
8,TA,TA,No,Unf,0,Unf,0,952,952,0,0
13,Gd,TA,Av,Unf,0,Unf,0,1494,1494,0,0
15,TA,TA,No,Unf,0,Unf,0,832,832,0,0
17,NaN,NaN,NaN,NaN,0,NaN,0,0,0,0,0
20,Ex,TA,Av,Unf,0,Unf,0,1158,1158,0,0
21,TA,TA,No,Unf,0,Unf,0,637,637,0,0
22,Gd,TA,No,Unf,0,Unf,0,1777,1777,0,0
25,Gd,TA,No,Unf,0,Unf,0,1566,1566,0,0
29,TA,TA,No,Unf,0,Unf,0,520,520,0,0
30,TA,TA,No,Unf,0,Unf,0,649,649,0,0


In [26]:
def compare_df(data1, data2, column_name):
    data1 = data1.filter(regex = column_name).iloc[:,0]
    df1 = pd.DataFrame(data1.value_counts(normalize=True) * 100)
    
    data2 = data2.filter(regex = column_name).iloc[:,0]
    df2 = pd.DataFrame(data2.value_counts(normalize=True) * 100)
    
    df3 = pd.merge(df1,df2, how='outer', left_index=True, right_index=True)
    
    return df3

train_bsmt_cat = train_bsmt.select_dtypes(exclude=["number","bool_"])

for col in train_bsmt_cat.columns:
    print(compare_df(train_bsmt,train,col))

    BsmtQual_x  BsmtQual_y
Ex    7.906977    8.503162
Fa    6.046512    2.459592
Gd   43.720930   43.429375
TA   42.325581   45.607871
    BsmtCond_x  BsmtCond_y
Fa    5.813953    3.162333
Gd    3.720930    4.567814
Po    0.232558    0.140548
TA   90.232558   92.129304
    BsmtExposure_x  BsmtExposure_y
Av       10.722611       15.541491
Gd        2.564103        9.423347
Mn        6.293706        8.016878
No       80.419580       67.018284
     BsmtFinType1_x  BsmtFinType1_y
ALQ             NaN       15.460295
BLQ             NaN       10.400562
GLQ             NaN       29.374561
LwQ             NaN        5.200281
Rec             NaN        9.346451
Unf           100.0       30.217850
     BsmtFinType2_x  BsmtFinType2_y
ALQ             NaN        1.336146
BLQ             NaN        2.320675
GLQ             NaN        0.984529
LwQ             NaN        3.234880
Rec             NaN        3.797468
Unf           100.0       88.326301


In [27]:
train['BsmtFinSF1'] = np.where(train['BsmtFinSF1'] <= 0, -1,train['BsmtFinSF1'])
train = train.drop(train[train['BsmtFinSF1'] > 2000].index)

**Variable**: BsmtFinSF2  
**Comments**: ~88.0% are zeros  
**Findings**: This tells us that BsmtFinSF2 is zero when the Basement2 is unfinished. I will change it to -1

In [28]:
train_bsmt = train[bsmt_cols][train['BsmtFinSF2']== 0]
for col in train_bsmt_cat.columns:
    print(compare_df(train_bsmt,train,col))

    BsmtQual_x  BsmtQual_y
Ex    9.025559    8.245243
Fa    2.715655    2.466526
Gd   45.287540   43.551797
TA   42.971246   45.736434
    BsmtCond_x  BsmtCond_y
TA   92.412141   92.107118
Gd    3.993610    4.580691
Fa    3.434505    3.171247
Po    0.159744    0.140944
    BsmtExposure_x  BsmtExposure_y
No       67.945643       67.207334
Av       15.587530       15.514810
Gd        8.792966        9.238364
Mn        7.673861        8.039492
     BsmtFinType1_x  BsmtFinType1_y
Unf       34.345048       30.303030
GLQ       30.990415       29.175476
ALQ       13.258786       15.503876
BLQ        8.945687       10.429880
Rec        8.785942        9.372798
LwQ        3.674121        5.214940
     BsmtFinType2_x  BsmtFinType2_y
ALQ             NaN        1.339915
BLQ             NaN        2.327221
GLQ             NaN        0.987306
LwQ             NaN        3.244006
Rec             NaN        3.808181
Unf           100.0       88.293371


In [29]:
train['BsmtFinSF2'] = np.where(train['BsmtFinSF2'] <= 0, -1,train['BsmtFinSF2'])

**Variable**: BsmtFullBath    
**Comments**: ~59.0% are zeros  
**Findings**: Some of them are zeros when there is no basement (I would have to change those rows to -1 for columns BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath, BsmtHalfBath

In [30]:
bsmt_col_to_chng = ['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']
train[train['BsmtQual'].isnull()][bsmt_col_to_chng] = train[train['BsmtQual'].isnull()][bsmt_col_to_chng].replace({0:np.nan})

**Variable**: BsmtHalfBath    
**Comments**: >90.0% are zeros  
**Findings**: This column will go into the drop list

In [31]:
col_to_rem.append('BsmtHalfBath')
col_to_rem

['3SsnPorch', 'BsmtHalfBath']

**Variable**: BsmtUnfSF  
**Comments**: Looks like the distribution is not normal. I will try to use the log and compare the before and after  
**Findings**: The distribution is a lot more normalized with the log

In [32]:
pd.set_option('mode.chained_assignment', None)
train['BsmtUnfSF_log'] = np.log(train['BsmtUnfSF']+1)

In [33]:
trace0 = go.Histogram(
    x=train['BsmtUnfSF_log'],
    name='BsmtUnfSF_log',
    nbinsx = 100,
    marker=dict(
        color='grey',
        line = dict(
            color = 'black',
            width = 0.5
          ),
    ),
    opacity = 0.70,
  )
trace1 = go.Histogram(
    x=train['BsmtUnfSF'],
    name='BsmtUnfSF',
    nbinsx = 100,
    marker=dict(
        color='blue',
        line = dict(
            color = 'black',
            width = 0.5
          ),
    ),
    opacity = 0.70
  )
  
fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('BsmtUnfSF_log','BsmtUnfSF'))
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



**Variable**: EnclosedPorch  
**Comments**: Looks like there are a lot of zeros which makes sense because not a lot of houses would have an enclosed porch

**Variable**: Fireplaces  
**Comments**: Looks like ~40% of them are zeros which makes sense because not a lot of houses would have a fireplace

**Variable**: FullBath  
**Comments**: The variable looks clean with only 0.6% as zeros

**Variable**: GarageArea  
**Comments**: There are no zeros here 

## Model Development

In [34]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb